In [1]:
import cv2
import numpy as np
import csv

def canny_edge(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    return cv2.Canny(blur, 50, 150)

def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([
        [(100, height), (image.shape[1] - 100, height), (image.shape[1] // 2, int(height * 0.6))]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    return cv2.bitwise_and(image, mask)

def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 10)
    return line_image

def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1 * 0.6)
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return np.array([x1, y1, x2, y2])

def average_slope_intercept(image, lines):
    left, right = [], []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope, intercept = parameters
        if slope < 0:
            left.append(parameters)
        else:
            right.append(parameters)
    if len(left) == 0 or len(right) == 0:
        return np.array([])
    left_avg = np.average(left, axis=0)
    right_avg = np.average(right, axis=0)
    left_line = make_coordinates(image, left_avg)
    right_line = make_coordinates(image, right_avg)
    return np.array([left_line, right_line])

def calculate_steering_angle(image, lines):
    height, width = image.shape[0], image.shape[1]
    if len(lines) == 0:
        return 0
    left_line, right_line = lines
    mid = width // 2
    x_offset = ((left_line[2] + right_line[2]) // 2) - mid
    y_offset = height // 2
    angle = np.degrees(np.arctan2(y_offset, x_offset))
    return int(angle)

# ---- Main Execution ----

video_path = "test_video.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Cannot open video file {video_path}")
    exit()

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames in video: {total_frames}")

frame_count = 0

# CSV writer setup
csv_file = open('steering_output.csv', mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Frame', 'Steering Angle (degrees)'])

# Video writer setup
fourcc = cv2.VideoWriter_fourcc(*'XVID')
ret, test_frame = cap.read()
out = cv2.VideoWriter('output_video.avi', fourcc, 20.0, (test_frame.shape[1], test_frame.shape[0]))
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # reset to frame 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    print(f"\nProcessing frame {frame_count} of {total_frames}")

    canny = canny_edge(frame)
    cropped = region_of_interest(canny)
    lines = cv2.HoughLinesP(cropped, 2, np.pi / 180, 100, np.array([]), minLineLength=40, maxLineGap=5)

    if lines is not None:
        averaged_lines = average_slope_intercept(frame, lines)
        if len(averaged_lines) > 0:
            line_image = display_lines(frame, averaged_lines)
            combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
            angle = calculate_steering_angle(frame, averaged_lines)
            print(f"Predicted steering angle: {angle}°")
            csv_writer.writerow([frame_count, angle])
        else:
            combo_image = frame
            print("No clear lane lines detected.")
            csv_writer.writerow([frame_count, 'N/A'])
    else:
        combo_image = frame
        print("No lines detected.")
        csv_writer.writerow([frame_count, 'N/A'])

    # Write frame to output video
    out.write(combo_image)

    # Show result
    cv2.imshow("Lane Detection", combo_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Final cleanup
cap.release()
out.release()
csv_file.close()
cv2.destroyAllWindows()


Total frames in video: 221

Processing frame 1 of 221
Predicted steering angle: 90°

Processing frame 2 of 221
Predicted steering angle: 90°

Processing frame 3 of 221
Predicted steering angle: 90°

Processing frame 4 of 221
Predicted steering angle: 90°

Processing frame 5 of 221
No clear lane lines detected.

Processing frame 6 of 221
No clear lane lines detected.

Processing frame 7 of 221
No clear lane lines detected.

Processing frame 8 of 221
No clear lane lines detected.

Processing frame 9 of 221
No clear lane lines detected.

Processing frame 10 of 221
No clear lane lines detected.

Processing frame 11 of 221
Predicted steering angle: 91°

Processing frame 12 of 221
Predicted steering angle: 90°

Processing frame 13 of 221
Predicted steering angle: 90°

Processing frame 14 of 221
Predicted steering angle: 91°

Processing frame 15 of 221
Predicted steering angle: 90°

Processing frame 16 of 221
Predicted steering angle: 90°

Processing frame 17 of 221
No clear lane lines detect

In [2]:
import pandas as pd

df = pd.read_csv("./steering_output.csv")
print(df.head())


   Frame  Steering Angle (degrees)
0      1                      90.0
1      2                      90.0
2      3                      90.0
3      4                      90.0
4      5                       NaN
